In [19]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def show_image(title, image):
    plt.figure(figsize=(10, 10))
    plt.title(title)
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.show()

def adjust_brightness(image, target_brightness=128, threshold=30):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    avg_brightness = np.mean(gray)
    beta = target_brightness - avg_brightness
    if abs(beta) > threshold:
        new_image = cv2.convertScaleAbs(image, alpha=1.0, beta=beta)
    else:
        new_image = image
    return new_image

def detect_and_fix_brightness(image_path, target_brightness=128, threshold=30):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    avg_brightness = np.mean(gray)
    print(f"Average brightness: {avg_brightness}")
    if abs(target_brightness - avg_brightness) > threshold:
        print("Brightness adjustment needed.")
        image = adjust_brightness(image, target_brightness, threshold)
        _, gray_binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        show_image("Binarized Grayscale Image", gray_binary)
    else:
        print("Brightness is within acceptable range.")
    return image

def process_image(image_path, output_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to read image from {image_path}")
        return

    # Step 1: Detect and fix brightness if needed
    image = detect_and_fix_brightness(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 2: Apply Sobel filter to detect edges
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(grad_x, grad_y)
    sobel = cv2.convertScaleAbs(sobel)

    # Step 3: Sharpen the image to enhance lines
    blurred = cv2.GaussianBlur(sobel, (5, 5), 0)
    sharpened = cv2.addWeighted(sobel, 2.0, blurred, -1.0, 0)

    # Step 4: Apply binary threshold
    _, binary = cv2.threshold(sharpened, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Step 5: Morphological closing to connect barcode lines
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 3))
    closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    
    # Step 6: Invert the image (invert the binary image for barcode detection)
    inverted = cv2.bitwise_not(closed)
    
    # Step 7: Binarize Grayscale Image
    _, gray_binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Show intermediate and final results
    show_image("Original Grayscale", gray)
    show_image("Sobel Edge Detection", sobel)
    show_image("Sharpened Image", sharpened)
    show_image("Binary Thresholded", binary)
    show_image("Inverted Image", inverted)
    show_image("Binarized Grayscale Image", gray_binary)
    
    # Save the final processed image
    cv2.imwrite(output_path, gray_binary)  # Save the filled image
    print(f"Processed image saved to {output_path}")


In [23]:
# Example usage
#process_image('C://Uni//Senior 1 Sem1//CV//Project//barcode_vision//Test Case Images//05 - meen taffa el nour!!!.jpg', 'cropped_image.jpg')
#process_image('C://Uni//Senior 1 Sem1//CV//Project//barcode_vision//Test Case Images//06 - meen fata7 el nour 333eenaaayy.jpg', 'cropped_image.jpg')
detect_and_fix_brightness('C://Uni//Senior 1 Sem1//CV//Project//barcode_vision//Test Case Images//05 - meen taffa el nour!!!.jpg')
#process_image('C://Uni//Senior 1 Sem1//CV//Project/barcode_vision//Test Case Images//09 - e3del el soora ya3ammm.jpg','cropped_image.jpg')
#detect_and_fix_brightness('C://Uni//Senior 1 Sem1//CV//Project//barcode_vision//Test Case Images//07 - mal7 w felfel.jpg')

Average brightness: 157.6206875
Brightness is within acceptable range.


array([[[232, 232, 232],
        [255, 255, 255],
        [133, 133, 133],
        ...,
        [255, 255, 255],
        [ 87,  87,  87],
        [248, 248, 248]],

       [[115, 115, 115],
        [ 93,  93,  93],
        [197, 197, 197],
        ...,
        [ 74,  74,  74],
        [ 62,  62,  62],
        [200, 200, 200]],

       [[ 92,  92,  92],
        [168, 168, 168],
        [108, 108, 108],
        ...,
        [154, 154, 154],
        [101, 101, 101],
        [180, 180, 180]],

       ...,

       [[208, 208, 208],
        [ 85,  85,  85],
        [ 80,  80,  80],
        ...,
        [ 71,  71,  71],
        [241, 241, 241],
        [255, 255, 255]],

       [[255, 255, 255],
        [108, 108, 108],
        [186, 186, 186],
        ...,
        [238, 238, 238],
        [ 65,  65,  65],
        [246, 246, 246]],

       [[238, 238, 238],
        [230, 230, 230],
        [208, 208, 208],
        ...,
        [255, 255, 255],
        [252, 252, 252],
        [200, 200, 200]]